In [ ]:
import os
import numpy as np
import pandas as pd
import mne 


# fifFileMddPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/mdd1624.fif"
# fifFileHcPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/hc1624.fif"

fifFileHcEpochPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif"
fifFileMddEpochPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif"   

allFeatureDirPath = "/Users/wachiii/Workschii/brain-mdd/data/balancedfeatures"
# mddRaw = mne.io.read_raw_fif(fifFileMddPath)
# hcRaw = mne.io.read_raw_fif(fifFileHcPath)
# print(mddRaw)
# print(hcRaw)

## Preprocessing
- bandpass filter 0.5 - 40 Hz
- Epoching for 10 sec => 10*250 = 2500 samples

In [ ]:
# do preprocessing: bandpass filter 0.5 - 40 Hz
mddEeg = mddRaw.load_data()
hcEeg = hcRaw.load_data()

mddEeg.filter(0.5, 40, fir_design='firwin')
hcEeg.filter(0.5, 40, fir_design='firwin')

# plot psd only 1 channel
mddEeg.plot_psd(fmin=0.5, fmax=40, average=True, spatial_colors=True) 
hcEeg.plot_psd(fmin=0.5, fmax=40, average=True, spatial_colors=True)

# do epoching; fs = 250, total samples = 1784136 where 1784136/24 = 74339.0 samples per subject -> i want epoch for 10 seconds
# 10 seconds = 2500 samples

mddEpochs = mne.make_fixed_length_epochs(mddEeg, duration=10, preload=True)
hcEpochs = mne.make_fixed_length_epochs(hcEeg, duration=10, preload=True)

print(mddEpochs)
print(hcEpochs)

# mddEpochs.save("/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif", overwrite=True)
# hcEpochs.save("/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif", overwrite=True)

In [ ]:
74339.0/(10*250)

## Feature Extraction

### 1. Band Power - PSD

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.psd import EEGPowerSpectrum
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }    

In [ ]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)

print(len(mddEpochs))
print(len(hcEpochs))

print(mddEpochs.get_data()[0].shape)
print(hcEpochs.get_data()[0].shape)

In [ ]:
psdObjMdd = EEGPowerSpectrum(mddEpochs, 250, ifNormalize=False)
mddPsdFeatures = psdObjMdd.run()
psdObjCtrl = EEGPowerSpectrum(hcEpochs, 250, ifNormalize=False)
hcPsdFeatures = psdObjCtrl.run()
print(mddPsdFeatures.shape)
print(hcPsdFeatures.shape)

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/bandpower/mddBandpowerFeatures.npy", mddPsdFeatures)
np.save(allFeatureDirPath + "/bandpower/hcBandpowerFeatures.npy", hcPsdFeatures)

### 2. Relative Power

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.psd import EEGPowerSpectrum
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }    

In [ ]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)

print(len(mddEpochs))
print(len(hcEpochs))

print(mddEpochs.get_data()[0].shape)
print(hcEpochs.get_data()[0].shape)

In [ ]:
psdObjMdd = EEGPowerSpectrum(mddEpochs, 250, ifNormalize=True)
mddPsdFeatures = psdObjMdd.run()
psdObjCtrl = EEGPowerSpectrum(hcEpochs, 250, ifNormalize=True)
hcPsdFeatures = psdObjCtrl.run()
print(mddPsdFeatures.shape)
print(hcPsdFeatures.shape)

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/relativepower/mddRelativepowerFeatures.npy", mddPsdFeatures)
np.save(allFeatureDirPath + "/relativepower/hcRelativepowerFeatures.npy", hcPsdFeatures)

### 3. HFD

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.hfd import EEGHiguchiFractalDimension
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "HFD": {
        "mdd": [],
        "control": []
    }
}

In [ ]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

In [ ]:
hfdObjMdd = EEGHiguchiFractalDimension(mddEpochs)
mddHfdFeatures = hfdObjMdd.run()
hfdObjControl = EEGHiguchiFractalDimension(hcEpochs)
hcHfdFeatures = hfdObjControl.run()

print(mddHfdFeatures.shape)
print(hcHfdFeatures.shape)

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/hfd/mddHfdFeatures.npy", mddHfdFeatures)
np.save(allFeatureDirPath + "/hfd/hcHfdFeatures.npy", hcHfdFeatures)

### 4. Coherences

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.coherence import EEGCoherence
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9 
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "coherence": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    } 


In [ ]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
mddEpochs = mddEpochs.get_data()
# print paired channels
coherenceObjMdd = EEGCoherence(mddEpochs, MODMA_SFREQ)
print(coherenceObjMdd.channel_pairs)

In [ ]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

In [ ]:
coherenceObjMdd = EEGCoherence(mddEpochs, MODMA_SFREQ)
mddCohFeatures = coherenceObjMdd.run()
coherenceObjControl = EEGCoherence(hcEpochs, MODMA_SFREQ)
hcCohFeatures = coherenceObjControl.run()

print(mddCohFeatures.shape)
print(hcCohFeatures.shape)

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/coh/mddCohFeatures.npy", mddCohFeatures)
np.save(allFeatureDirPath + "/coh/hcCohFeatures.npy", hcCohFeatures)

### 5. Cd

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.correlationdimension import EEGCorrelationDimension
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "CD": {
        "mdd": [],
        "control": []
    }
}

In [ ]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

In [ ]:
cdObjMdd = EEGCorrelationDimension(mddEpochs)
mddCdFeatures = cdObjMdd.run()
cdObjControl = EEGCorrelationDimension(hcEpochs)
hcCdFeatures = cdObjControl.run()

for epoch in mddCdFeatures:
    featureDict["CD"]["mdd"].append(epoch)
for epoch in hcCdFeatures:
    featureDict["CD"]["control"].append(epoch)

In [ ]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/cd/mddCdFeatures.npy", mddCdFeatures)
np.save(allFeatureDirPath + "/cd/hcCdFeatures.npy", hcCdFeatures)

## Feature Selection & ML

In [24]:
import os
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import dotenv
import warnings

dotenv.load_dotenv()
warnings.filterwarnings("ignore")


allFeatureDirPath = os.getenv("BALANCED_FEATURES_DIR_PATH")
featureSelectionModelDirPath = os.getenv("FEATURE_SELECTION_MODEL_DIR_PATH")

hcPsdFeatures = np.load(allFeatureDirPath + "/bandpower/hcBandpowerFeatures.npy")
mddPsdFeatures = np.load(allFeatureDirPath + "/bandpower/mddBandpowerFeatures.npy")
hcRelativepowerFeatures = np.load(allFeatureDirPath + "/relativepower/hcRelativepowerFeatures.npy")
mddRelativepowerFeatures = np.load(allFeatureDirPath + "/relativepower/mddRelativepowerFeatures.npy")
hcCohFeatures = np.load(allFeatureDirPath + "/coh/hcCohFeatures.npy")
mddCohFeatures = np.load(allFeatureDirPath + "/coh/mddCohFeatures.npy")
hcHfdFeatures = np.load(allFeatureDirPath + "/hfd/hcHfdFeatures.npy")
mddHfdFeatures = np.load(allFeatureDirPath + "/hfd/mddHfdFeatures.npy")
hcCdFeatures = np.load(allFeatureDirPath + "/cd/hcCdFeatures.npy")
mddCdFeatures = np.load(allFeatureDirPath + "/cd/mddCdFeatures.npy")

print(f"hc psd feature shape: {hcPsdFeatures.shape}")
print(f"mdd psd feature shape: {mddPsdFeatures.shape}")
print(f"relative power hc feature shape: {hcRelativepowerFeatures.shape}")
print(f"relative power mdd feature shape: {mddRelativepowerFeatures.shape}")
print(f"hc coh feature shape: {hcCohFeatures.shape}")
print(f"mdd coh feature shape: {mddCohFeatures.shape}")
print(f"hc hfd feature shape: {hcHfdFeatures.shape}")
print(f"mdd hfd feature shape: {mddHfdFeatures.shape}")
print(f"hc cd feature shape: {hcCdFeatures.shape}")
print(f"mdd cd feature shape: {mddCdFeatures.shape}")

hc psd feature shape: (713, 5, 16)
mdd psd feature shape: (713, 5, 16)
relative power hc feature shape: (713, 5, 16)
relative power mdd feature shape: (713, 5, 16)
hc coh feature shape: (713, 5, 120)
mdd coh feature shape: (713, 5, 120)
hc hfd feature shape: (713, 16)
mdd hfd feature shape: (713, 16)
hc cd feature shape: (713, 16)
mdd cd feature shape: (713, 16)


In [25]:
# Flatten features to make the shape consistent
hcPsdFeatures = hcPsdFeatures.reshape(hcPsdFeatures.shape[0], -1)  # Flatten to (samples, features)
hcRelativepowerFeatures = hcRelativepowerFeatures.reshape(hcRelativepowerFeatures.shape[0], -1)
hcCohFeatures = hcCohFeatures.reshape(hcCohFeatures.shape[0], -1)
hcHfdFeatures = hcHfdFeatures.reshape(hcHfdFeatures.shape[0], -1)
hcCdFeatures = hcCdFeatures.reshape(hcCdFeatures.shape[0], -1)

mddPsdFeatures = mddPsdFeatures.reshape(mddPsdFeatures.shape[0], -1)
mddRelativepowerFeatures = mddRelativepowerFeatures.reshape(mddRelativepowerFeatures.shape[0], -1)
mddCohFeatures = mddCohFeatures.reshape(mddCohFeatures.shape[0], -1)
mddHfdFeatures = mddHfdFeatures.reshape(mddHfdFeatures.shape[0], -1)
mddCdFeatures = mddCdFeatures.reshape(mddCdFeatures.shape[0], -1)

print(f"hc psd feature shape: {hcPsdFeatures.shape}")
print(f"mdd psd feature shape: {mddPsdFeatures.shape}")
print(f"relative power hc feature shape: {hcRelativepowerFeatures.shape}")
print(f"relative power mdd feature shape: {mddRelativepowerFeatures.shape}")
print(f"hc coh feature shape: {hcCohFeatures.shape}")
print(f"mdd coh feature shape: {mddCohFeatures.shape}")
print(f"hc hfd feature shape: {hcHfdFeatures.shape}")
print(f"mdd hfd feature shape: {mddHfdFeatures.shape}")
print(f"hc cd feature shape: {hcCdFeatures.shape}")
print(f"mdd cd feature shape: {mddCdFeatures.shape}")

hc psd feature shape: (713, 80)
mdd psd feature shape: (713, 80)
relative power hc feature shape: (713, 80)
relative power mdd feature shape: (713, 80)
hc coh feature shape: (713, 600)
mdd coh feature shape: (713, 600)
hc hfd feature shape: (713, 16)
mdd hfd feature shape: (713, 16)
hc cd feature shape: (713, 16)
mdd cd feature shape: (713, 16)


In [26]:
models = {
    "KNN": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7, 9], "weights": ["uniform", "distance"]}),
    "SVM": (SVC(), {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}),
    "Decision Tree": (DecisionTreeClassifier(), {"max_depth": [5, 10, 15]}),
    "Random Forest": (RandomForestClassifier(), {"n_estimators": [50, 100], "max_depth": [10, 20]}),
    "Logistic Regression": (LogisticRegression(), {"C": [0.01, 0.1, 1]})
}

featureSelectors = {
    "NoFeatureSelection": None,
    "SelectKBest": SelectKBest(score_func=f_classif, k=100),
    "VarianceThreshold": VarianceThreshold(threshold=0.01)  # Removes low variance features
    # "RFE": RFE(LogisticRegression(), n_features_to_select=100)
}

def get_stratified_kfold_data(X, y, nSplits=5):
    skf = StratifiedKFold(n_splits=nSplits, shuffle=True, random_state=42)
    return skf.split(X, y)

In [27]:
hcFeatures = np.concatenate([
            hcPsdFeatures, hcRelativepowerFeatures, hcCohFeatures, hcHfdFeatures, hcCdFeatures
            ], axis=1)  
mddFeatures = np.concatenate([
            mddPsdFeatures, mddRelativepowerFeatures, mddCohFeatures, mddHfdFeatures, mddCdFeatures
            ], axis=1)



X = np.concatenate([hcFeatures, mddFeatures], axis=0)
y = np.concatenate([
    np.zeros(hcPsdFeatures.shape[0]),
    np.ones(mddPsdFeatures.shape[0])   
])

print(f"X shape: {X.shape}")  
print(f"y shape: {y.shape}")

X shape: (1426, 792)
y shape: (1426,)


In [ ]:
# Initialize totals for MDD and HC counts
totalHcTrain = 0
totalMddTrain = 0
totalHcVal = 0
totalMddVal = 0
totalHcTest = 0
totalMddTest = 0

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

np.save(os.path.join(featureSelectionModelDirPath, "X_test.npy"), X_test)
np.save(os.path.join(featureSelectionModelDirPath, "y_test.npy"), y_test)

results = []

for modelName, (model, paramGrid) in models.items():
    for featureSelectorName, featureSelector in featureSelectors.items():
        print(f"Training {modelName} with {featureSelectorName}...")
        if featureSelector is not None:
            selector = featureSelector
        else:
            selector = None
        foldAccuracies = []
        foldPrecisions = []
        foldRecalls = []
        foldF1Scores = []
        selectedFeatures = []
        foldHcTrain = []
        foldMddTrain = []
        foldHcVal = []
        foldMddVal = []
        foldHcTest = []
        foldMddTest = []

        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        for fold, (trainIdx, valIdx) in enumerate(skf.split(X_train_val, y_train_val)):
            XTrain, XVal = X_train_val[trainIdx], X_train_val[valIdx]
            yTrain, yVal = y_train_val[trainIdx], y_train_val[valIdx]
            
            hcTrain = int(np.sum(yTrain == 0))
            mddTrain = int(np.sum(yTrain == 1))
            hcVal = int(np.sum(yVal == 0))
            mddVal = int(np.sum(yVal == 1))
            hcTest = int(np.sum(y_test == 0))
            mddTest = int(np.sum(y_test == 1))
            
            foldHcTrain.append(hcTrain)
            foldMddTrain.append(mddTrain)
            foldHcVal.append(hcVal)
            foldMddVal.append(mddVal)
            foldHcTest.append(hcTest)
            foldMddTest.append(mddTest)

            if selector is not None:
                XTrain = selector.fit_transform(XTrain, yTrain)
                XVal = XVal[:, selector.get_support()]
            
            model.fit(XTrain, yTrain)
            
            yPred = model.predict(XVal)
            foldAccuracies.append(accuracy_score(yVal, yPred))
            foldPrecisions.append(precision_score(yVal, yPred))
            foldRecalls.append(recall_score(yVal, yPred))
            foldF1Scores.append(f1_score(yVal, yPred))
            
            if selector is not None:
                selectedFeatures.append(np.where(selector.get_support())[0].tolist())  # Store as list

            model_dir = os.path.join(featureSelectionModelDirPath, f"{modelName}_{featureSelectorName}")
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)
            
            model_filename = os.path.join(model_dir, f"model_fold_{fold + 1}.pkl")
            joblib.dump(model, model_filename)
            print(f"Model for fold {fold + 1} saved to {model_filename}")

        avgAccuracy = np.mean(foldAccuracies)
        avgPrecision = np.mean(foldPrecisions)
        avgRecall = np.mean(foldRecalls)
        avgF1Score = np.mean(foldF1Scores)
        
        flatSelectedFeatures = [item for sublist in selectedFeatures for item in sublist]  # Flatten the list
        significantFeatures = np.unique(flatSelectedFeatures)  # Unique selected features across all folds

        totalHcTrain += np.sum(foldHcTrain)
        totalMddTrain += np.sum(foldMddTrain)
        totalHcVal += np.sum(foldHcVal)
        totalMddVal += np.sum(foldMddVal)
        totalHcTest += np.sum(foldHcTest)
        totalMddTest += np.sum(foldMddTest)

        results.append({
            "model": modelName,
            "feature_selection": featureSelectorName,
            "avg_accuracy": avgAccuracy,
            "avg_precision": avgPrecision,
            "avg_recall": avgRecall,
            "avg_f1_score": avgF1Score,
            "significant_features": significantFeatures.tolist(),  
            "fold_hc_train": [int(x) for x in foldHcTrain], 
            "fold_mdd_train": [int(x) for x in foldMddTrain],  
            "fold_hc_val": [int(x) for x in foldHcVal], 
            "fold_mdd_val": [int(x) for x in foldMddVal],  
            "fold_hc_test": [int(x) for x in foldHcTest],  
            "fold_mdd_test": [int(x) for x in foldMddTest]  
        })

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(featureSelectionModelDirPath, "results.csv"), index=False)
results_df.to_excel(os.path.join(featureSelectionModelDirPath, "results.xlsx"), index=False)

for result in results:
    print(f"\n{result['model']} with {result['feature_selection']} - Fold-wise counts")
    for i in range(5):
        print(f"Fold {i + 1}: HC Train = {result['fold_hc_train'][i]}, MDD Train = {result['fold_mdd_train'][i]}, "
              f"HC Val = {result['fold_hc_val'][i]}, MDD Val = {result['fold_mdd_val'][i]}, "
              f"HC Test = {result['fold_hc_test'][i]}, MDD Test = {result['fold_mdd_test'][i]}")

print(f"\nTotal HC in training: {totalHcTrain} (Total: {X_train_val.shape[0]})")
print(f"Total MDD in training: {totalMddTrain} (Total: {X_train_val.shape[0]})")
print(f"Total HC in validation: {totalHcVal} (Total: {X_train_val.shape[0]})")
print(f"Total MDD in validation: {totalMddVal} (Total: {X_train_val.shape[0]})")
print(f"Total HC in test: {totalHcTest} (Total: {X_test.shape[0]})")
print(f"Total MDD in test: {totalMddTest} (Total: {X_test.shape[0]})")

Training KNN with NoFeatureSelection...
Model for fold 1 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_1.pkl
Model for fold 2 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_2.pkl
Model for fold 3 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_3.pkl
Model for fold 4 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_4.pkl
Model for fold 5 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_5.pkl
Model for fold 6 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_6.pkl
Model for fold 7 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_selection/KNN_NoFeatureSelection/model_fold_7.pkl
Model for fold 8 saved to /Users/wachiii/Workschii/brain-mdd/models/feature_se

In [ ]:
channelLst = ["Fp1", "Fp2", "F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2", "F7", "F8", "T3", "T4", "T5", "T6"]
bands = ["Delta", "Theta", "Alpha", "Beta", "Gamma"]

psdFeatureNames = [f'PSD_{ch}_{band}' for ch in channelLst for band in bands]
relativePowerFeatureNames = [f'RelativePower_{ch}_{band}' for ch in channelLst for band in bands]
coh_pairs = [(channelLst[i], channelLst[j]) for i in range(len(channelLst)) for j in range(i+1, len(channelLst))]
cohFeatureNames = [f'Coherence_{ch1}_{ch2}_{band}' for ch1, ch2 in coh_pairs for band in bands]
hfdFeatureNames = [f'HFD_{ch}' for ch in channelLst]
cdFeatureNames = [f'CD_{ch}' for ch in channelLst]
allFeatureNames = (
    psdFeatureNames + relativePowerFeatureNames + cohFeatureNames + hfdFeatureNames + cdFeatureNames
)
print(f"Total number of features: {len(allFeatureNames)}") 

# import json
# feature_dict = {feature_name: index+1 for index, feature_name in enumerate(allFeatureNames)}

# with open('feature_names.json', 'w') as json_file:
#     json.dump(feature_dict, json_file, indent=4)

Total number of features: 792
